# Building Spin-Liqud in Python

## Libraries

In [3]:
from quspin.operators import hamiltonian,quantum_operator # operators
from quspin.basis import spin_basis_general # spin basis constructor
import numpy as np # general math functions

## Parameters, basis and symmetries

In [27]:
Lx, Ly = 5, 2# linear dimension of spin 1 2d lattice
N2d = Lx*Ly  #number of sites for spin 1/2
basis = spin_basis_general(N2d,pauli=0)
no_checks = dict(check_symm=False,check_herm=False,check_pcon=False)

In [168]:
#(np.zeros([3,3]) != 0).any()
spin_basis_general(N2d)
600/60

10.0

## Hamiltonian

In [20]:
def Kitaev_H(alpha = 1, S = None , Js = [1.0,1.0,1.0], J_coup = -1.0 ):
    ''' This function computes the hamiltonian of the Kitaev model coupled 
     to a general localized potentials.The kitaev model by default have a
     size of 5x2, it can be generalized later
    Args:
    ----
    alpha : 1D float, optional 
    This parameter in the kitaev Hamiltonian, it determines the phase 
    
    S : dictionary 
    Contains the local potential to be coupled to each one of the
    spin operators. 
    
    Js: 1D array of floats
    Contains the coupling between each one of the spin components, e.g.
    Js=[J_x, J_y, J_z] -> H = J_xS_x S_x + J_yS_yS_y ....
    j_coup: is the strenght between between the system and the locali_
    zed potentials 
    '''
    # Couplings between spin 
    Jxx,Jyy,Jzz = Js
    # Dictionary that contains the elements 
    sigma_i = S
    # Configugarion 5x2 npbc
    J_yy = [[Jyy,0,1],[Jyy,2,3],[Jyy,6,7],[Jyy,8,9]]
    J_xx = [[Jxx,1,2],[Jxx,3,4],[Jxx,5,6],[Jxx,7,8]]
    J_zz = [[Jzz,5,0],[Jzz,2,7],[Jzz,4,9]]
    # if S is not None:          
    #     J_x = [ [sigma_i[f'Sigma_x_{1}']  ,    6], [sigma_i[f'Sigma_x_{2}']  ,    7]  , [sigma_i[f'Sigma_x_{3}']  ,    8]           ]
    #     J_y = [ [sigma_i[f'Sigma_y_{1}']  ,    6], [sigma_i[f'Sigma_y_{2}']  ,    7]  , [sigma_i[f'Sigma_y_{3}']  ,    8]           ]
    #     J_z = [ [sigma_i[f'Sigma_z_{1}']  ,    6], [sigma_i[f'Sigma_z_{2}']  ,    7]  , [sigma_i[f'Sigma_z_{3}']  ,    8]           ]
    #     operator_list_3 = [["z", J_z  ], ['x', J_x  ],  ['y', J_y  ]    ]
    # Kitaev elements of the couplings
    operator_list_0 = [["zz",J_zz],["yy",J_yy],["xx",J_xx]]
    # Contain the Heinsenberg elements of each coupling 
    operator_list_1 = [["xx",J_zz],["yy",J_zz],
                      ["xx",J_yy],["zz",J_yy],
                      ["yy",J_xx],["zz",J_xx]]
    # if S is not None:
    #     # Operators to be used
    #     operator_dict = dict(H0=operator_list_0,H1=operator_list_1,Hcoup=operator_list_3)        
    #     # Parameters that acompains each operator 
    #     params_dict = dict(H0 = np.sin(alpha)+np.cos(alpha)
    #                      ,H1 = np.cos(alpha), Hcoup = J_coup)        
    # else:     
    operator_dict = dict(H0=operator_list_0,H1 = operator_list_1)
    params_dict = dict(H0=np.sin(alpha)+np.cos(alpha),H1=np.cos(alpha))
    # Build the hamiltonian in quspin form 
    H = quantum_operator(operator_dict,basis=basis,**no_checks).tohamiltonian(params_dict)
    # Put the parameters in the hamiltonian 
    H_1 = H.tohamiltonian(params_dict)
    return  H_1     

In [21]:
#E_GS, psi_GS = Hamiltonian()

def spin_op(basis=basis, sites=(0,1) ): 
    site1,site2=sites
    sigma = [['zz', [[1.0, site1,site2]]] ,
             ['yy', [[1.0, site1,site2]]],
             ['xx', [[1.0, site1,site2]]]]
    return hamiltonian(sigma,dynamic,
                       dtype=np.float64,basis=basis,**no_checks)
def spin_op_z(basis = basis,sites = 1 ):#sites=(0,1) ):
    site1 = sites
    sigma = [['z', [[1.0, site1]]] ]
    return hamiltonian(sigma,dynamic,
                       dtype=np.complex128,basis=basis,**no_checks)
def spin_op_x(basis=basis, sites=1 ):
    site1=sites
    sigma = [['x', [[1.0, site1]]] ]
    return hamiltonian(sigma,dynamic,
                       dtype=np.complex128,basis=basis,**no_checks)
def spin_op_y(basis=basis, sites=1 ):
    site1=sites
    sigma = [['y', [[1.0, site1]]   ]   ] 
    return hamiltonian(sigma,dynamic,
                       dtype=np.complex128,basis=basis,**no_checks)
def Kitaev_correlations(eve):
    correlations=[]
    correlation1=spin_op(sites=(3,2) ).expt_value(eve,check=False)
    correlation2=spin_op(sites=(1,2) ).expt_value(eve,check=False)
    correlation3=spin_op(sites=(7,2) ).expt_value(eve,check=False)
    correlation=correlation1+correlation2+correlation3
    #print('step')
    return correlation

In [154]:
Lx, Ly = 5, 2# linear dimension of spin 1 2d lattice
N2d = Lx*Ly  #number of sites for spin 1/2
basis = spin_basis_general(N2d,pauli=1)
no_checks = dict(check_symm=False,check_herm=False,check_pcon=False)
HBAR = 0.658211928e0
### Hamiltonian
def Kitaev_H(alpha = 1, S = np.zeros([3,3]) , Js = [1.0,1.0,1.0], J_coup = 1.0 ):
    '''This function computes the hamiltonian of the Kitaev model coupled 
    to a general localized potentials.The kitaev model by default have a
    size of 5x2, it can be generalized later
    Args:
    ----
    alpha : 1D float, optional 
    This parameter in the kitaev Hamiltonian, it determines the phase 
    S : dictionary 
    Contains the local potential to be coupled to each one of the
    spin operators. 
    Js: 1D array of floats
    Contains the coupling between each one of the spin components, e.g.
    Js=[J_x, J_y, J_z] -> H = J_xS_x S_x + J_yS_yS_y ....
    j_coup: is the strenght between between the system and the locali_
    zed potentials 
    '''
    ##print("join0 " ,S )
    # Couplings between spin 
    Jxx,Jyy,Jzz = Js                                ### Magnitude of the coupling 
    # Configugarion 5x2 npbc
    J_yy = [[Jyy,0,1],[Jyy,2,3],[Jyy,6,7],[Jyy,8,9]]
    J_xx = [[Jxx,1,2],[Jxx,3,4],[Jxx,5,6],[Jxx,7,8]]
    J_zz = [[Jzz,5,0],[Jzz,2,7],[Jzz,4,9]]
    operator_list_0 = [["zz",J_zz],["yy",J_yy],["xx",J_xx]]
    # Contain the Heinsenberg elements of each coupling 
    J_yxx = [[Jxx,0,1],[Jxx,2,3],[Jxx,6,7],[Jxx,8,9]]
    J_yzz = [[Jzz,0,1],[Jzz,2,3],[Jzz,6,7],[Jzz,8,9]]
    J_xyy = [[Jyy,1,2],[Jyy,3,4],[Jyy,5,6],[Jyy,7,8]]
    J_xzz = [[Jzz,1,2],[Jzz,3,4],[Jzz,5,6],[Jzz,7,8]]
    J_zxx = [[Jxx,5,0],[Jxx,2,7],[Jxx,4,9]]
    J_zyy = [[Jyy,5,0],[Jyy,2,7],[Jyy,4,9]]
    operator_list_1 = [["xx",J_zxx],["yy",J_zyy],
                      ["xx",J_yxx],["zz",J_yzz],
                      ["yy",J_xyy],["zz",J_xzz]]
    operator_dict = dict(H0=operator_list_0,H1 = operator_list_1)
    ### Parameter that controls the HK model 
    params_dict = dict(H0=np.sin(alpha)+np.cos(alpha),H1=np.cos(alpha))
    #(np.zeros([3,3]) !== 0).any()
    #[:,1]
    # print("bool_val :",(S != 0).any())
    if (S != 0.).any() :
        ##print("join " ,S )
        J_x = [ [S[0,0] , 6], [S[0,1]  , 7], [S[0,2],  8]  ]
        J_y = [ [S[1,0] , 6], [S[1,1]  , 7], [S[1,2],  8]  ]
        J_z = [ [S[2,0] , 6], [S[2,1]  , 7], [S[2,2],  8]  ]
        operator_list_3 = [["z", J_z  ], ['x', J_x  ], ['y', J_y ] ]
        operator_dict = dict(H0=operator_list_0,H1=operator_list_1,Hcoup=operator_list_3)
        params_dict = dict(H0=np.sin(alpha)+np.cos(alpha),H1=np.cos(alpha), Hcoup = -J_coup)

    ##print(operator_dict)
    ##print(params_dict)
    # Build the hamiltonian in quspin form 
    H = quantum_operator(operator_dict,basis=basis,dtype=np.complex128,**no_checks)
    # Put the parameters in the hamiltonian 
    H_1 = H.tohamiltonian(params_dict)
    return  H_1     

### Operators 

def spin_op(basis=basis, sites=(0,1) ): 
    site1,site2=sites
    sigma = [['zz', [[1.0, site1,site2]]] ,
             ['yy', [[1.0, site1,site2]]],
             ['xx', [[1.0, site1,site2]]]]
    return hamiltonian(sigma,[],
                       dtype=np.complex128,basis=basis,**no_checks)
def spin_op_z(basis = basis,sites = 1 ):#sites=(0,1) ):
    site1 = sites
    sigma = [['z', [[1.0, site1]]] ]
    return hamiltonian(sigma,[],
                       dtype=np.complex128,basis=basis,**no_checks)
def spin_op_x(basis=basis, sites=1 ):
    site1=sites
    sigma = [['x', [[1.0, site1]]] ]
    return hamiltonian(sigma,[],
                       dtype=np.complex128,basis=basis,**no_checks)
def spin_op_y(basis=basis, sites=1 ):
    site1=sites
    sigma = [['y', [[1.0, site1]]   ]   ] 
    return hamiltonian(sigma,[],
                       dtype=np.complex128,basis=basis,**no_checks)

def spindensity_qsl(psi,sites=[5,6,7]):
    sden= []
    for site in sites:
        S_x = spin_op_x(sites = site).expt_value(V = psi)#.toarray()
        S_y = spin_op_y(sites = site).expt_value(V = psi)#.toarray()
        S_z = spin_op_z(sites = site).expt_value(V = psi)#.toarray()
        sden.append([S_x, S_y, S_z])
    return sden

def Kitaev_correlations(eve):
    correlations=[]
    correlation1=spin_op(sites=(3,2) ).expt_value(eve,check=False)
    correlation2=spin_op(sites=(1,2) ).expt_value(eve,check=False)
    correlation3=spin_op(sites=(7,2) ).expt_value(eve,check=False)
    correlation=correlation1+correlation2+correlation3
    #print('step')
    return correlation

   

In [172]:
E_S, psi_S = np.linalg.eig(Kitaev_H(alpha=np.pi, Js = [1.,1.,1.],J_coup = 0.0 ).toarray()) ### eigh tiene problmes
#E_S, psi_S = Kitaev_H(alpha=np.pi, Js = [0.,0.,1.],J_coup = 0.0 ).eigh()#eigsh(k=1,which="SA",maxiter=1E4)#.eigh()
#psi_S = eigen(Kf.Kitaev_H(alpha=pi, Js = [0.,1.,0.],J_coup = 0.0 ).toarray() ).vectors
#Kf.Kitaev_H(alpha=pi, Js = [0.,0.,1.],J_coup = 0.0 ).eigh()#.eigsh(k=30,which="SA",maxiter=1E8)
#np.linalg.eigh(Kf.Kitaev_H(alpha=pi, Js = [0.,1.,0.],J_coup = 0.0 ).toarray())
##.eigh()#.eigsh(k=1,which="SA",maxiter=1E4)#.eigh()#.eigsh(k = 1,which = "SA")#.eigh()#.eigsh(k = 1,which = "SA") #.eigh()#.eigsh(which = "SA")#
psi_GS = psi_S[:,0] 
#Kf.basis.ent_entropy(psi_GS ,sub_sys_A=A,alpha=1)["Sent_A"][1]
A=[0,1,2,3,4]
ssden = []
for i in range(9):
    S_z = spin_op_z(sites = i).expt_value(V = psi_GS).real#.toarray()
    S_x = spin_op_x(sites = i).expt_value(V = psi_GS).real#.toarray()
    S_y = spin_op_y(sites = i).expt_value(V = psi_GS).real#.toarray()
    #S_eq.append([np.real(S_x_v), np.real(S_y_v), np.real(S_z_v)])
    ssden.append( [S_x,S_y,S_z] ) 
    print([S_x,S_y,S_z]  )
#### Check if the GS is FM
ennt= basis.ent_entropy(psi_GS ,sub_sys_A=A,alpha=1,density=True)["Sent_A"]#[0]
ennt

[0.0, 0.0, 1.0]
[0.0, 0.0, 1.0]
[0.0, 0.0, 1.0]
[-8.274590478674932e-51, 1.3177747429038154e-82, 1.0]
[0.0, 0.0, 1.0]
[0.0, 0.0, 1.0]
[0.0, 0.0, 1.0]
[0.0, 0.0, 1.0]
[7.447505915230829e-50, 2.5246406103036556e-67, 1.0]


array(4.95760435e-14)

In [143]:
#np.linalg.norm(psi_GS)

# E_S, psi_S = Kitaev_H(alpha=1.).eigh()#.eigsh(which = "SA")#.eigsh(k = 1,which = "SA") 
# psi_GS = psi_S[:,1] ;

# spindensity_qsl(psi=psi_GS)#[1,:]
#Kitaev_H(alpha=np.pi, Js = [1.,1.,1.],J_coup = 0.0 ).toarray()

In [24]:
# site1 = 0
# dynamic = []
# sigma = [['z', [[1.0, site1]]] ]
# sx = hamiltonian(sigma,dynamic,dtype=np.complex128,basis=basis,**no_checks)

In [16]:
# # Define parameters
# L = 6  # Number of spins
# J = 1.0  # Exchange coupling constant

# # Create spin basis
# basis = spin_basis_general(L,pauli=0)

# # Define Heisenberg Hamiltonian
# J_matrix = [[J, i, (i + 1) % L] for i in range(L)]
# static = [["xx", J_matrix], ["yy", J_matrix], ["zz", J_matrix]]
# H = hamiltonian(static, [], basis=basis, dtype=np.float64)

# # Diagonalize the Hamiltonian
# E, psi = H.eigh()
# A=[0,1,2,3,4]
# #print("Eigenenergies:", E)
# basis.ent_entropy(psi[1,:] ,sub_sys_A=A,alpha=1)["Sent_A"]#[1]

Hermiticity check passed!
Symmetry checks passed!


array(0.12730283)

In [15]:
#psi[:,1]

array([ 0.00000000e+00,  0.00000000e+00, -4.75857457e-16, -1.12837010e-01,
        5.58448221e-16,  2.95411127e-01, -1.12837010e-01,  4.28121595e-05,
        1.38777878e-17, -3.65148234e-01,  2.95411127e-01, -1.12083689e-04,
       -1.12837010e-01, -1.12083689e-04,  4.28121595e-05,  2.64913776e-07,
       -5.82231105e-16,  2.95411127e-01, -3.65148234e-01, -1.12083689e-04,
        2.95411127e-01,  5.44065654e-04, -1.12083689e-04, -6.93553269e-07,
       -1.12837010e-01, -1.12083689e-04, -1.12083689e-04,  8.57278986e-07,
        4.28121595e-05, -6.93553269e-07,  2.64913776e-07,  0.00000000e+00,
        4.75857457e-16, -1.12837010e-01,  2.95411127e-01,  4.28121595e-05,
       -3.65148234e-01, -1.12083689e-04, -1.12083689e-04,  2.64913776e-07,
        2.95411127e-01, -1.12083689e-04,  5.44065654e-04, -6.93553269e-07,
       -1.12083689e-04,  8.57278986e-07, -6.93553269e-07, -1.72313225e-26,
       -1.12837010e-01,  4.28121595e-05, -1.12083689e-04,  2.64913776e-07,
       -1.12083689e-04, -